Title of the Analysis
---

Short description of what's being done
* Data:
* Aim:
* Challenges: 

# Getting Started

Here, we load in all required packages, the data and if necesssary, additional annotations.

## Load Packages

Load in all relevant packages and set the python paths. Configure plotting parameters, set a directory for figures, results and the data.

In [ ]:
# only want to see warning once
import warnings
warnings.filterwarnings(action='once') 

In [ ]:
# import standard packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
import seaborn as sns
from tqdm import tqdm
import scipy
import os
import sys
import re

In [ ]:
# set the path to the home directory of this project
base_path = os.path.dirname(os.getcwd())
base_path

In [ ]:
# import scanpy, scvelo and velocyto
import scanpy.api as sc
import scvelo as scv

In [ ]:
# logging
scv.logging.print_versions()

In [ ]:
import sys  
sys.path.insert(0, '/opt/projects/interactive_plotting/src')  

import interactive_plotting as ipl  
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# parameters for scvelo and scanpy
scv.settings.set_figure_params(frameon = True, style='scvelo')
sc.settings.verbosity = 0

## Set up Paths for Writing and Reading

In [ ]:
# set up paths for writing adata files, for importing data and 
# for saving figuresb
create_dirs = True

In [ ]:
# check the figures directories
scv.settings.figdir, sc.settings.figdir

In [ ]:
np.random.seed(42)

## Import Data

In [ ]:
# import data
adata = sc.datasets.paul15()

In [ ]:
# make unique
adata.var_names_make_unique()

In [ ]:
# cell and gene numbers
adata

In [ ]:
# spliced and unspliced numbers
scv.utils.show_proportions(adata)

## Additional Annotations

Import additionall annotations. Ideally, the path would be stored in ```paths```, or given relative to the ```base_path```.

In [ ]:
# label mitochondtial genes
regex = re.compile('^(mt).*', re.IGNORECASE)
mito_genes = [l for l in adata.var_names for m in [regex.search(l)] if m]
adata.var['mito'] = False
adata.var.loc[mito_genes, 'mito'] = True
print('Found {} mito genes and annotated.'.format(len(mito_genes)))

In [ ]:
mito_genes[:5]

In [ ]:
# label batches
adata.obs['batch'] = '1'
adata.obs.loc[adata.obs_names[:int(adata.n_obs/2)], 'batch'] = '0'

In [ ]:
# compute qc metrics
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], inplace=True)
adata.obs.head()

## Quality Control

In [ ]:
# start with looking at counts in the raw adata object
sc.pl.highest_expr_genes(adata)

In [ ]:
# check out quality metric across batches
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mito'], groupby='batch'

In [ ]:
adata.obs['group'] = np.random.choice(['group_1', 'group_2'], adata.n_obs)
adata.obs['group'] = adata.obs['group'].astype('category')

adata.obs['batch'] = np.random.choice(['batch_1', 'batch_2'], adata.n_obs)
adata.obs['batch'] = adata.obs['batch'].astype('category')

adata.obs['plate'] = np.random.choice(['plate_1', 'plate_2', 'plate_3'], adata.n_obs)
adata.obs['plate'] = adata.obs['plate'].astype('category')

In [ ]:
# visualise as histograms
ipl.interactive_hist(adata, groups=['plate'],
                     keys=['n_genes_by_counts', 'total_counts', 'pct_counts_mito'], 
                     fill_alpha=0.3,
                     plot_width=400, plot_height=400)

In [ ]:
?ipl.thresholding_hist

In [ ]:
# visualise as histograms
ipl.thresholding_hist(adata, key='n_genes', categories=dict(foo=[0, 500], bar=[1000, 1200]),
                      bases=['umap'],
                      plot_width=400, plot_height=400)

In [ ]:
# plot percentage of mitochondtial genes versus count depth and n_genes
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='pct_counts_mito')

## Filtering

We will filter out the human and the viral genes, and we will look at some quality measures for the actual mouse genes.

In [ ]:
# basic filtering of cells and genes
sc.pp.filter_cells(adata, min_genes=200)
# filter based on spliced/unspliced reads per cell
scv.pp.filter_genes(adata, min_counts=20, min_counts_u=10, min_cells=3)

In [ ]:
# filter cells based on total counts/n_genes
adata = adata[adata.obs['total_counts'] < 8000].copy()

In [ ]:
# filter cells based on mitochondtial activity
adata = adata[adata.obs['pct_counts_mito'] < 2]

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='pct_counts_mito')

## Normalisation

In [ ]:
# normalize per cell - we use scvelo here to normalize to initial cell size
scv.pp.normalize_per_cell(adata, use_initial_size=True)

In [ ]:
# log transform and set .raw
adata.raw = sc.pp.log1p(adata, copy=True)
sc.pp.log1p(adata)

In [ ]:
# batch effect removal
sc.pp.combat(adata, key='batch')

In [ ]:
# filter on highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=3000)

In [ ]:
# plot highly variable genes
sc.pl.highly_variable_genes(adata)

In [ ]:
# regress out the count depth effect
sc.pp.regress_out(adata, 'total_counts')

In [ ]:
#scale the genes so that they equaly contribute to the PCA
sc.pp.scale(adata)

# Embedding and Clustering

Typical workflow:
- PCA, choose number of components
- Quality assesment in the PCA space
- Compute a KNN graph
- Compute UMAP, force directed layout, tSNE, etc
- Run Louvain or Leiden clustering

In [ ]:
# Comptue PCA representation
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# Look at variance distribution
sc.pl.pca_variance_ratio(adata)

In [ ]:
# set a number of pc's to be used in downstream analysis
n_pcs = 11

In [ ]:
# Look at quality metrics in the PCA plot
sc.pl.pca(adata, color=['total_counts', 'n_genes_by_counts', 'pct_counts_mito'])

In [ ]:
# Check the loadings
sc.pl.pca_loadings(adata)

**Interpretetation:**

In [ ]:
# compute the neighborhood graph
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=n_pcs, random_state=42)

In [ ]:
# comptue clustering
sc.tl.louvain(adata, resolution=0.2, random_state=42)

In [ ]:
# compute UMAP
sc.tl.umap(adata, random_state=42)

In [ ]:
# plot the umap
sc.pl.umap(adata, color=['louvain', 'total_counts', 'n_genes_by_counts', 'pct_counts_mito'])

## Differential expression testing

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain', groups=['1', '2', '3'], reference='0')
ipl.highlight_de(adata, cell_keys='batch')

## DPT

In [ ]:
sc.tl.diffmap(adata)
ipl.highlight_indices(adata, key='group', basis='diffmap', components=[1, 4])

In [ ]:
adata.uns['iroot'] = 2371
sc.tl.dpt(adata)

In [ ]:
sc.pl.diffmap(adata, color='dpt_pseudotime', components=[1, 4])

In [ ]:
ipl.link_plot(adata, bases=['diffmap', 'umap'], components=[[1, 4], [1, 2]],
              genes=list(map(lambda r: r[0], adata.uns['rank_genes_groups']['names']))[:10],
              cutoff=True, # highlight_only='louvain',
              key='louvain', distance='dpt', legend_loc=None)

## Velocity

In [ ]:
ipl.velocity_plot(adata, genes=adata.var_names[:2], paths=[['0', '3', '5'], ['0', '3', '6']])